

---
- MAI XUÂN BÁCH
- 11200489
- DSEB 62


---



---




**QUESTIONS**

Cho 2 pretrained model trên 2 dataset CIFAR-10 và MNIST, finetune trên dataset mới FashionMNIST (có trong thư viện torchvision).

- Data augmentation: normalize ảnh từ scale [0, 255] về [-1,1], sử dụng ít nhất 2 trong số các transformations dưới đây cho dataset: Random resized, Center cropping, Random vertical flipping, Random horizontal flipping, Các loại transformation khác có thể tham khảo ở đây https://pytorch.org/vision/master/transforms.html

- Fine-tuning: Cho mô hình MiniVGG như dưới, hãy load 2 pretrained models vào model MiniVGG ở dưới. Freeze self.features và train lớp cuối (self.classifier) của 2 pretrained models trên dataset FashionMNIST. Sau đó, train model MiniVGG from scratch trên tập FashionMNIST.

Với 3 models, train 5 epochs và báo cáo accuracy trên tập test của 3 models Mô hình nào có accuracy cao nhất và tại sao? Nếu train với nhiều epochs hơn thì accuracy của 3 models có giống nhau không?

- Feature extractor: Với model tương tự như homework 2, dùng get_graph_node_names() và create_feature_extractor từ thư viện torchvision.models.feature_extraction để in ra tên layer và weight của layer tương ứng

**IMPORTS**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision import models

# 1. Data augmentation

In [ ]:
transform = transforms.Compose([
    transforms.Resize((28,28)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)) # Normalize to [-1, 1]
])

In [ ]:
# Load dataset
train_data = torchvision.datasets.FashionMNIST(
    root="./data", train=True, download=True, transform=transform
)
test_data = torchvision.datasets.FashionMNIST(
    root="./data", train=False, download=True, transform=transform
)

# Create dataloaders
train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=64, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=64, shuffle=False
)

100%|██████████| 26421880/26421880 [00:02<00:00, 11146575.82it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 201686.35it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3742477.54it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 5963070.14it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#cifar_10_pre_model = models.vgg11(pretrained = True)
#cifar_10_pre_model.load_state_dict(torch.load("cifar10_mini_vgg.pth"))

In [ ]:
class MiniVGG(nn.Module):
    def __init__(self,):
        super(MiniVGG, self).__init__()
        self.features = nn.Sequential(nn.Conv2d(1,64,kernel_size=3,padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.Conv2d(64,64,kernel_size=3,padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.MaxPool2d(kernel_size=2,stride=2),

                                      nn.Conv2d(64,128,kernel_size=3,padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.Conv2d(128,128,kernel_size=3,padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.MaxPool2d(kernel_size=2,stride=2),

                                      nn.Conv2d(128,256,kernel_size=3,padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.Conv2d(256,256,kernel_size=3,padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.MaxPool2d(kernel_size=2, stride=2),


        )
        self.classifier = nn.Linear(256*3*3,10)
        nn.init.normal_(self.classifier.weight,0, 0.01)
        nn.init.constant_(self.classifier.bias,0)


    def forward(self,x,):
        x = self.features(x)
        x = torch.flatten(x,1)
        x = self.classifier(x)
        return x

# 2. Finetuning

In [ ]:
model = MiniVGG().to(device)
model.load_state_dict(torch.load("cifar10_mini_vgg.pth", map_location=torch.device('cpu')))
for param in model.parameters():
    param.requires_grad = False
for param in model.classifier.parameters():
    param.requires_grad = True

In [ ]:
model_2 = MiniVGG().to(device)
model_2.load_state_dict(torch.load('mnist_mini_vgg.pth',map_location=torch.device('cpu')))
for param in model_2.parameters():
    param.requires_grad = False
for param in model_2.classifier.parameters():
    param.requires_grad = True

In [ ]:
model_3 = MiniVGG().to(device)

In [ ]:
model

MiniVGG(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Linear(in_features=2304, out_features=10, bias=True)
)

In [ ]:
model_2

MiniVGG(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Linear(in_features=2304, out_features=10, bias=True)
)

In [ ]:
model_3

MiniVGG(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Linear(in_features=2304, out_features=10, bias=True)
)

In [ ]:
lr = 0.001
moment = 0.9

In [ ]:
# Loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=moment)

# Set the batch size of the model
#model.batch_size = 3

for epoch in range(5):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        #loss.requires_grad = True
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        #loss.requires_grad = True
        optimizer.step()

        if (i + 1) % 100 == 0:
            print("Epoch: {}/5, Step: {}/{}, Loss: {:.4f}".format(
                epoch + 1, i + 1, len(train_loader), loss.item()
            ))

model.eval()
with torch.no_grad():
    total_correct = 0
    total_samples = 0

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        batch_size = images.size(0)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    accuracy = total_correct / total_samples

    print(f'Learning Rate: {lr}, Accuracy: {accuracy}')

Epoch: 1/5, Step: 100/938, Loss: 0.8716
Epoch: 1/5, Step: 200/938, Loss: 0.8412
Epoch: 1/5, Step: 300/938, Loss: 0.8546
Epoch: 1/5, Step: 400/938, Loss: 0.4254
Epoch: 1/5, Step: 500/938, Loss: 0.6400
Epoch: 1/5, Step: 600/938, Loss: 0.3884
Epoch: 1/5, Step: 700/938, Loss: 0.3127
Epoch: 1/5, Step: 800/938, Loss: 0.5258
Epoch: 1/5, Step: 900/938, Loss: 0.4565
Epoch: 2/5, Step: 100/938, Loss: 0.6036
Epoch: 2/5, Step: 200/938, Loss: 0.3947
Epoch: 2/5, Step: 300/938, Loss: 0.3940
Epoch: 2/5, Step: 400/938, Loss: 0.7728
Epoch: 2/5, Step: 500/938, Loss: 0.4083
Epoch: 2/5, Step: 600/938, Loss: 0.2919
Epoch: 2/5, Step: 700/938, Loss: 0.4520
Epoch: 2/5, Step: 800/938, Loss: 0.2921
Epoch: 2/5, Step: 900/938, Loss: 0.2435
Epoch: 3/5, Step: 100/938, Loss: 0.5518
Epoch: 3/5, Step: 200/938, Loss: 0.3183
Epoch: 3/5, Step: 300/938, Loss: 0.1869
Epoch: 3/5, Step: 400/938, Loss: 0.3964
Epoch: 3/5, Step: 500/938, Loss: 0.3415
Epoch: 3/5, Step: 600/938, Loss: 0.4410
Epoch: 3/5, Step: 700/938, Loss: 0.6094


In [ ]:
# Loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_2.parameters(), lr=lr, momentum=moment)

# Set the batch size of the model
model_2.batch_size = 3

for epoch in range(5):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model_2(images)
        #loss.requires_grad = True
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        # loss.requires_grad = True
        loss.backward()
        #loss.requires_grad = True
        optimizer.step()

        if (i + 1) % 100 == 0:
            print("Epoch: {}/5, Step: {}/{}, Loss: {:.4f}".format(
                epoch + 1, i + 1, len(train_loader), loss.item()
            ))


model_2.eval()
with torch.no_grad():
    total_correct = 0
    total_samples = 0

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        batch_size = images.size(0)
        outputs = model_2(images)
        _, predicted = torch.max(outputs.data, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    accuracy = total_correct / total_samples

    print(f'Learning Rate: {lr}, Accuracy: {accuracy}')


Epoch: 1/5, Step: 100/938, Loss: 1.4329
Epoch: 1/5, Step: 200/938, Loss: 0.6004
Epoch: 1/5, Step: 300/938, Loss: 0.3417
Epoch: 1/5, Step: 400/938, Loss: 0.4598
Epoch: 1/5, Step: 500/938, Loss: 0.5689
Epoch: 1/5, Step: 600/938, Loss: 0.6485
Epoch: 1/5, Step: 700/938, Loss: 0.3370
Epoch: 1/5, Step: 800/938, Loss: 0.3816
Epoch: 1/5, Step: 900/938, Loss: 0.5827
Epoch: 2/5, Step: 100/938, Loss: 0.3452
Epoch: 2/5, Step: 200/938, Loss: 0.6238
Epoch: 2/5, Step: 300/938, Loss: 0.4112
Epoch: 2/5, Step: 400/938, Loss: 0.4004
Epoch: 2/5, Step: 500/938, Loss: 0.3255
Epoch: 2/5, Step: 600/938, Loss: 0.4075
Epoch: 2/5, Step: 700/938, Loss: 0.3818
Epoch: 2/5, Step: 800/938, Loss: 0.6509
Epoch: 2/5, Step: 900/938, Loss: 0.4844
Epoch: 3/5, Step: 100/938, Loss: 0.5788
Epoch: 3/5, Step: 200/938, Loss: 0.4892
Epoch: 3/5, Step: 300/938, Loss: 0.5256
Epoch: 3/5, Step: 400/938, Loss: 0.6352
Epoch: 3/5, Step: 500/938, Loss: 0.2651
Epoch: 3/5, Step: 600/938, Loss: 0.4807
Epoch: 3/5, Step: 700/938, Loss: 0.4662


In [ ]:
# Loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_3.parameters(), lr=lr, momentum=moment)


for epoch in range(5):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model_3(images)
        #loss.requires_grad = True
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        # loss.requires_grad = True
        loss.backward()
        #loss.requires_grad = True
        optimizer.step()

        if (i + 1) % 100 == 0:
            print("Epoch: {}/5, Step: {}/{}, Loss: {:.4f}".format(
                epoch + 1, i + 1, len(train_loader), loss.item()
            ))

model_3.eval()
with torch.no_grad():
    total_correct = 0
    total_samples = 0

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        batch_size = images.size(0)
        outputs = model_3(images)
        _, predicted = torch.max(outputs.data, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    accuracy = total_correct / total_samples

    print(f'Learning Rate: {lr}, Accuracy: {accuracy}')

Epoch: 1/5, Step: 100/938, Loss: 2.3013
Epoch: 1/5, Step: 200/938, Loss: 2.2990
Epoch: 1/5, Step: 300/938, Loss: 2.2989
Epoch: 1/5, Step: 400/938, Loss: 2.2939
Epoch: 1/5, Step: 500/938, Loss: 2.2867
Epoch: 1/5, Step: 600/938, Loss: 2.1318
Epoch: 1/5, Step: 700/938, Loss: 1.2196
Epoch: 1/5, Step: 800/938, Loss: 0.8483
Epoch: 1/5, Step: 900/938, Loss: 0.7755
Epoch: 2/5, Step: 100/938, Loss: 1.0395
Epoch: 2/5, Step: 200/938, Loss: 0.7631
Epoch: 2/5, Step: 300/938, Loss: 0.6003
Epoch: 2/5, Step: 400/938, Loss: 0.6598
Epoch: 2/5, Step: 500/938, Loss: 0.5493
Epoch: 2/5, Step: 600/938, Loss: 0.6259
Epoch: 2/5, Step: 700/938, Loss: 0.4880
Epoch: 2/5, Step: 800/938, Loss: 0.6186
Epoch: 2/5, Step: 900/938, Loss: 0.5061
Epoch: 3/5, Step: 100/938, Loss: 0.7053
Epoch: 3/5, Step: 200/938, Loss: 0.5312
Epoch: 3/5, Step: 300/938, Loss: 0.6016
Epoch: 3/5, Step: 400/938, Loss: 0.5265
Epoch: 3/5, Step: 500/938, Loss: 0.6219
Epoch: 3/5, Step: 600/938, Loss: 0.3862
Epoch: 3/5, Step: 700/938, Loss: 0.4990


Mô hình nào có accuracy cao nhất là: **Mô hình lấy pretrained trên cifar-10**.

Lí do: Dataset CIFAR có nhiều feature gần giống với dataset FashionMNIST hơn so với dataset MNIST.

Nếu train với nhiều epochs hơn thì accuracy của 3 models có giống nhau không?

Nếu train thêm thì model MiniVGG train from scratch sẽ tốt hơn 2 model pretrain vì các layer sẽ fit được với FashionMNIST dataset tốt hơn. 2 model pretrain (các layer đã bị freeze) do vậy nếu train thêm nữa thì layer của model cũng sẽ không học được thêm

# 3. Feature Extraction

In [ ]:
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor

train_nodes, eval_nodes = get_graph_node_names(model_3)

In [ ]:
train_nodes

['x',
 'features.0',
 'features.1',
 'features.2',
 'features.3',
 'features.4',
 'features.5',
 'features.6',
 'features.7',
 'features.8',
 'features.9',
 'features.10',
 'features.11',
 'features.12',
 'features.13',
 'features.14',
 'flatten',
 'classifier']

In [ ]:
create_feature_extractor(model_3, train_return_nodes= train_nodes, eval_return_nodes= eval_nodes)

MiniVGG(
  (features): Module(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Linear(in_features=2304, out_features=10, bias=True)
)

In [ ]:
model_3.features[0].weight

Parameter containing:
tensor([[[[-2.8759e-01,  2.0110e-01,  2.0866e-01],
          [-1.4713e-02, -2.0285e-02, -2.5287e-01],
          [-1.7327e-01,  1.1902e-01,  3.2581e-01]]],


        [[[-1.1258e-01,  6.7882e-02, -1.8877e-01],
          [-3.5911e-01,  9.1008e-02, -1.3706e-01],
          [-2.4694e-01, -3.7895e-01, -3.0658e-01]]],


        [[[ 3.2548e-01,  2.5163e-01,  1.6485e-01],
          [-2.3129e-01,  1.2172e-02, -2.8688e-01],
          [-7.2736e-02, -5.9601e-03,  3.1775e-01]]],


        [[[-3.1088e-02,  2.5738e-01, -3.3810e-02],
          [-1.4852e-01, -1.7919e-01, -2.0031e-01],
          [-1.7278e-01, -1.2197e-01, -2.4061e-01]]],


        [[[-2.7563e-01,  1.8458e-01,  2.1452e-01],
          [ 2.7147e-02,  3.2068e-01, -1.4611e-01],
          [ 2.9314e-01, -2.1236e-01, -3.1851e-01]]],


        [[[-1.4604e-01,  6.9329e-02, -1.1402e-01],
          [ 2.7333e-01, -2.7895e-01,  1.3624e-01],
          [-2.7887e-01,  6.5921e-02, -3.3683e-01]]],


        [[[ 3.9302e-02, -1.8237e-01,